# Get data and packages

In [1]:
%%capture
%%bash
pip install -U pip dask[complete] numpy fsspec>=0.3.3 tqdm pyarrow fastparquet python-snappy
# aws s3 cp --recursive s3://xdss-public-datasets/demos/taxi_parquet datasets/taxi_parquet

In [1]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os

instance_type = 'c5d2xlarge' # change this
results_bucket = f"s3://vaex-sagemaker-demo/benchmarks" # change this

name = 'dask'
data_path = 'datasets/taxi_parquet/*'
output_file = f'{name}_{instance_type}_tmp.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {
    'run':[],
    'duration': [],
    'task': []   
}

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks=benchmarks):
    return pd.DataFrame.from_dict(benchmarks, orient='index').T

def persist():
    gc.collect()
    get_results(benchmarks).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, **kwargs):    
    for i in range(2):
        start_time = time.time()
        ret = f(df, **kwargs)
        benchmarks['duration'].append(time.time()-start_time)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]
          
def add_nan(name):
    for i in range(2):
        benchmarks['duration'].append(np.nan)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]

        

!mkdir -p results
!mkdir -p datasets
print(f"We test every benchmark twice and save both results")

We test every benchmark twice and save both results


# Benchmark

In [2]:
import dask.dataframe as dd
import numpy as np

# Load data
data_path = "datasets/taxi_parquet/"
data = dd.read_parquet(data_path, engine='pyarrow')

# print(f"size: {len(data.vendor_id)} with {len(data.columns)} columns")

In [10]:
def read_file_parquet(df=None):
    return dd.read_parquet(data_path, engine='pyarrow')

benchmark(read_file_parquet, df=data, name='read_file')

read_file took: 0.7170224189758301


0.7170224189758301

In [3]:
def count(df=None):
    # there is a dask bug - len(df) takes 20X time longer
    return len(df.vendor_id) 

benchmark(count, df=data, name='count')

count took: 43.89179968833923


43.89179968833923

In [ ]:
def mean(df):
    return df.fare_amount.mean().compute()

benchmark(mean, df=data, name='mean')

In [ ]:
def standard_deviation(df):
    return df.fare_amount.std().compute()

benchmark(standard_deviation, df=data, name='standard deviation')

To calculate the time when using two columns, we can't return the response since it will get into memroy and break, so we run a mean calculation on it, and then remove the time it took to run the mean.

In [ ]:
def mean_of_sum(df):
    return (df.fare_amount + df.trip_distance).mean().compute()

benchmark(mean_of_sum, df=data, name='sum columns mean')
add_nan('sum columns')

In [41]:
# lazy evaulation - instant
def product(df):
    return df.fare_amount * df.trip_distance

benchmark(product, df=data, name='product columns')

product columns took: 0.001043081283569336


0.001043081283569336

In [ ]:
def mean_of_product(df):
    return (df.fare_amount * df.trip_distance).mean().compute()

benchmark(mean_of_product, df=data, name='product columns mean')
add_nan('product columns')

In [ ]:
def new_column_mean(df):
    df['lazy'] = df.fare_amount * df.trip_distance
    return df['lazy'].mean().compute()
    
benchmark(new_column_mean, df=data, name='new column mean')    

In [ ]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret.mean().compute()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean')
add_nan('arithmetic operation')

In [29]:
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return ret

benchmark(complicated_arithmetic_operation, df=data, name='arithmetic operation')

arithmetic operation took: 0.01173710823059082


0.01173710823059082

In [30]:
df = complicated_arithmetic_operation(data)

In [ ]:
def value_counts(df):
    return df.passenger_count.value_counts().compute()

benchmark(value_counts, df=data, name='value counts')

In [ ]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg({'fare_amount': ['mean', 'std'], 
                                               'tip_amount': ['mean', 'std']
                                              }).compute()

benchmark(groupby_statistics, df=data, name='groupby statistics')

In [ ]:
other = groupby_statistics(data).persist()
other = other.set_index('passenger_count').persist()
data = data.set_index('passenger_count').persist()

In [ ]:
def join_count(df, other):
    return len(dd.merge(df, other, left_index=True, right_index=True))

benchmark(join_count, data, name='join count', other=other)

In [ ]:
# crashes
def join_data(df, other):
    return dd.merge(df, other, left_index=True, right_index=True)

benchmark(join_data, data, name='join', other=other)

## Filtered data

Dask is not build to run on filter data like you would normally, so we will apply the same strategy

In [ ]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

In [ ]:
expr_filter = (data.pickup_longitude > long_min)  & (data.pickup_longitude < long_max) & \
                  (data.pickup_latitude > lat_min)    & (data.pickup_latitude < lat_max) & \
                  (data.dropoff_longitude > long_min) & (data.dropoff_longitude < long_max) & \
                  (data.dropoff_latitude > lat_min)   & (data.dropoff_latitude < lat_max)

def filter_data(df):
    return df[expr_filter].compute()

benchmark(filter_data, data, name='filter data')

In [ ]:
filtered = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

In [ ]:
benchmark(mean, filtered, name='filtered mean')
benchmark(standard_deviation, filtered, name='filtered standard deviation')
benchmark(mean_of_sum, filtered, name ='filtered sum columns mean')
benchmarks['filtered sum columns'] =  np.nan
benchmark(mean_of_product, filtered, name ='filtered product columns mean')
add_nan('filtered product columns')
benchmark(mean_of_complicated_arithmetic_operation, filtered, name='filtered arithmetic operation mean')
add_nan('filtered arithmetic operation')
benchmark(value_counts, filtered, name ='filtered value counts')
benchmark(groupby_statistics, filtered, name='filtered groupby statistics')
other = groupby_statistics(filtered)
add_nan('filtered join')
benchmark(join_count, filtered, name='filtered join count', other=other)
print(name)
get_results()